# change_data(input_file,out_file,r) 정의 :

input_file의 데이터를 읽고 각 로고 데이터의 r 비율을 바꿔 out_file로 저장

- input_file : .mat 파일 이름 (경로 포함)

- output_file : . mat 파일 이름 (경로 포함)

- r : [0,1] 의 소수

In [49]:
# .mat 파일 읽어서 각 로고 데이터의 r 비율을 바꾼다.

def change_data(input_file,out_file,r):
    
    # import 모듈
    from scipy import io
    import numpy as np
    import copy
    
    # .mat 파일 읽기
    mat_file = io.loadmat(input_file)
    
    # .mat 파일 내용 읽기
    groundtruth=mat_file['groundtruth']
    
    file_mat=groundtruth['file'][0]
    bboxes_mat=groundtruth['bboxes'][0]
    name_mat=groundtruth['name'][0]
    
    # 로고 종류 담을 리스트
    logo_lst=[]
    
    # 바꾼 데이터 개수
    num=0
    
    # 로고 종류 뽑기
    for logo in name_mat:

        if logo not in logo_lst:

            logo_lst.append(logo)

    # 각 로고마다 r 비율로 데이터 변형
    for logo in logo_lst:
        
        # 로고 수
        num_logo=sum(name_mat==logo)

        # 로고의 원래 인덱스
        # tuple 의미 있는 건 orig_ind[0]
        orig_ind=np.where(name_mat==logo)

        # 변형할 개수
        num_choice=(int)(num_logo*r)
        
        num+=num_choice

        # 변형할 데이터의 인덱스
        ind=np.random.choice(orig_ind[0],num_choice,replace=False)
        
        print(len(ind))

        # 위,왼쪽,아래쪽,오른쪽 변형할 개수 저장
        sides=np.array([0,0,0,0])

        # num_choice에서 4로 나누는데 나머지 있을 수 있으므로,
        # 고르게 하기 위해 다음과 같이 처리
        base= int(num_choice/4)
        
        sides=sides+base
        
        residue=int(num_choice%4)
        
        # 나머지를 어디에 더할지 랜덤으로 정함
        res=[0,1,2,3]

        choice=np.random.choice(res,residue,replace=False)

        sides[choice]+=1
        
        print(sides)
        
        print(sides[0])
        
        # ind 앞에서부터 차례대로 진행

        #'Up' side
        for i in range(sides[0]):
            
            print(ind)

            # 변형할 인덱스 저장하고 ind에서 지운다 
            indx=ind[0]
     
            # 바운딩 박스 여러 개일 수 있으므로 각각 적용 
            for bbox in bboxes_mat[indx]:

                # h
                bbox[3]/=2

                # y
                bbox[1]+=(bbox[3]/2)
        print(ind)       

        # 'Left' side
        for i in range(sides[1]):

            indx=ind[0]
            ind=np.delete(ind,0,0)

            for bbox in bboxes_mat[indx]:

                # w
                bbox[2]/=2

                # x
                bbox[0]-=(bbox[2]/2)
        print(ind)

        #'Down' side
        for i in range(sides[2]):


            indx=ind[0]
            ind=np.delete(ind,0,0)

            for bbox in bboxes_mat[indx]:

                # h
                bbox[3]/=2

                # y
                bbox[1]-=(bbox[3]/2)
        print(ind)

        #'Right' side
        for i in range(sides[3]):

            indx=ind[0]
            ind=np.delete(ind,0,0)
            
            for bbox in bboxes_mat[indx]:

                # w
                bbox[2]=bbox[2]/2

                # x
                bbox[0]+=(bbox[2]/2)

        print(ind)
    # 결과물 저장
    out=io.savemat(out_file,{'groundtruth':groundtruth})
    
    return num
    

# 실행 하기 :

In [ ]:
num=change_data('groundtruth.mat','h.mat',0.2)

33
[8 8 9 8]
8
[ 84  64  60  40  33 151  61  35  65  81  38 164 108 107  42  79  80 147
   3  56  86   8   4 102 163 101  16  39 143 111 154  67  43]
[ 84  64  60  40  33 151  61  35  65  81  38 164 108 107  42  79  80 147
   3  56  86   8   4 102 163 101  16  39 143 111 154  67  43]
[ 84  64  60  40  33 151  61  35  65  81  38 164 108 107  42  79  80 147
   3  56  86   8   4 102 163 101  16  39 143 111 154  67  43]
[ 84  64  60  40  33 151  61  35  65  81  38 164 108 107  42  79  80 147
   3  56  86   8   4 102 163 101  16  39 143 111 154  67  43]
[ 84  64  60  40  33 151  61  35  65  81  38 164 108 107  42  79  80 147
   3  56  86   8   4 102 163 101  16  39 143 111 154  67  43]
[ 84  64  60  40  33 151  61  35  65  81  38 164 108 107  42  79  80 147
   3  56  86   8   4 102 163 101  16  39 143 111 154  67  43]
[ 84  64  60  40  33 151  61  35  65  81  38 164 108 107  42  79  80 147
   3  56  86   8   4 102 163 101  16  39 143 111 154  67  43]
[ 84  64  60  40  33 151  61  35  65  81

28
[7 7 7 7]
7
[860 775 892 822 811 873 766 813 778 779 832 827 851 833 893 843 759 831
 762 848 834 877 829 761 760 846 824 874]
[860 775 892 822 811 873 766 813 778 779 832 827 851 833 893 843 759 831
 762 848 834 877 829 761 760 846 824 874]
[860 775 892 822 811 873 766 813 778 779 832 827 851 833 893 843 759 831
 762 848 834 877 829 761 760 846 824 874]
[860 775 892 822 811 873 766 813 778 779 832 827 851 833 893 843 759 831
 762 848 834 877 829 761 760 846 824 874]
[860 775 892 822 811 873 766 813 778 779 832 827 851 833 893 843 759 831
 762 848 834 877 829 761 760 846 824 874]
[860 775 892 822 811 873 766 813 778 779 832 827 851 833 893 843 759 831
 762 848 834 877 829 761 760 846 824 874]
[860 775 892 822 811 873 766 813 778 779 832 827 851 833 893 843 759 831
 762 848 834 877 829 761 760 846 824 874]
[860 775 892 822 811 873 766 813 778 779 832 827 851 833 893 843 759 831
 762 848 834 877 829 761 760 846 824 874]
[813 778 779 832 827 851 833 893 843 759 831 762 848 834 877 829 

[1722 1872 1944 1709 1881 1841 1875 1928 1765 1808 1892 1902 1776 1951
 1814 1732 1895 1761 1941 1850 1779 1769 1903 1945 1865 1697 1795 1691
 1707 1695 1934 1720 1880 1721 1715 1897 1763 1751 1686 1842]
[1951 1814 1732 1895 1761 1941 1850 1779 1769 1903 1945 1865 1697 1795
 1691 1707 1695 1934 1720 1880 1721 1715 1897 1763 1751 1686 1842]
[1691 1707 1695 1934 1720 1880 1721 1715 1897 1763 1751 1686 1842]
68
[17 17 17 17]
17
[2080 2072 2079 2194 2070 2252 2288 2199 2120 2186 2022 2286 1984 2183
 2203 2096 1992 2091 2197 1962 1972 2185 2236 2023 2258 2168 2116 2208
 2239 2061 2177 2202 1967 2121 2204 1990 2159 2210 2035 2065 2030 2274
 2014 2078 2151 1971 2109 2134 2212 2169 2243 2110 2058 2234 1976 2188
 2162 2218 2154 2057 2089 2060 1979 2233 2268 2238 2119 2144]
[2080 2072 2079 2194 2070 2252 2288 2199 2120 2186 2022 2286 1984 2183
 2203 2096 1992 2091 2197 1962 1972 2185 2236 2023 2258 2168 2116 2208
 2239 2061 2177 2202 1967 2121 2204 1990 2159 2210 2035 2065 2030 2274
 2014 2078 2

 2715 2824 2843 2873 2693 2854 2922 2838 2924 2860 2740 2878 2907]
[3009 2760 2988 2905 2984 2946 2780 2908 2997 2888 2840 2803 2799 2985
 2702 2892 2928 2781 2835 2899 2919 2719 2746 3023 2912 3018 2954 3028
 2861 2947 2832 2758 2752 2777 2849 2893 3004 3031 3019 2828 2698 2744
 2763 2756 2788 2797 2961 2992 2697 2754 2717 2938 2950 2772 2901 2883
 2715 2824 2843 2873 2693 2854 2922 2838 2924 2860 2740 2878 2907]
[3009 2760 2988 2905 2984 2946 2780 2908 2997 2888 2840 2803 2799 2985
 2702 2892 2928 2781 2835 2899 2919 2719 2746 3023 2912 3018 2954 3028
 2861 2947 2832 2758 2752 2777 2849 2893 3004 3031 3019 2828 2698 2744
 2763 2756 2788 2797 2961 2992 2697 2754 2717 2938 2950 2772 2901 2883
 2715 2824 2843 2873 2693 2854 2922 2838 2924 2860 2740 2878 2907]
[3009 2760 2988 2905 2984 2946 2780 2908 2997 2888 2840 2803 2799 2985
 2702 2892 2928 2781 2835 2899 2919 2719 2746 3023 2912 3018 2954 3028
 2861 2947 2832 2758 2752 2777 2849 2893 3004 3031 3019 2828 2698 2744
 2763 2756 2788 27

99
[24 25 25 25]
24
[4027 3853 3716 3740 3967 3877 3796 3702 3829 3797 3694 4123 3902 4091
 3963 4099 3849 3996 3836 3688 3788 3936 3833 3857 3760 3928 4036 3892
 3787 4070 4162 4121 3775 4014 3865 4084 3793 4076 4143 3732 3733 3846
 3908 3984 3695 3893 3850 3966 4152 3749 4003 3824 3790 3693 3800 3772
 4142 3779 4072 3685 4042 3825 3728 3978 3891 4097 3805 3753 4111 3923
 3926 3939 4158 3983 3980 3952 3914 3855 3985 3777 4100 3847 3764 3890
 3831 3792 3713 3721 3871 3712 3729 4128 4060 3842 4065 3677 3956 4174
 3868]
[4027 3853 3716 3740 3967 3877 3796 3702 3829 3797 3694 4123 3902 4091
 3963 4099 3849 3996 3836 3688 3788 3936 3833 3857 3760 3928 4036 3892
 3787 4070 4162 4121 3775 4014 3865 4084 3793 4076 4143 3732 3733 3846
 3908 3984 3695 3893 3850 3966 4152 3749 4003 3824 3790 3693 3800 3772
 4142 3779 4072 3685 4042 3825 3728 3978 3891 4097 3805 3753 4111 3923
 3926 3939 4158 3983 3980 3952 3914 3855 3985 3777 4100 3847 3764 3890
 3831 3792 3713 3721 3871 3712 3729 4128 4060 3842

45
[11 11 11 12]
11
[4759 4709 4726 4702 4794 4831 4631 4686 4724 4751 4634 4652 4712 4781
 4661 4784 4786 4684 4633 4656 4703 4741 4612 4796 4757 4701 4659 4657
 4632 4673 4826 4622 4677 4697 4824 4775 4692 4626 4705 4821 4787 4680
 4663 4750 4801]
[4759 4709 4726 4702 4794 4831 4631 4686 4724 4751 4634 4652 4712 4781
 4661 4784 4786 4684 4633 4656 4703 4741 4612 4796 4757 4701 4659 4657
 4632 4673 4826 4622 4677 4697 4824 4775 4692 4626 4705 4821 4787 4680
 4663 4750 4801]
[4759 4709 4726 4702 4794 4831 4631 4686 4724 4751 4634 4652 4712 4781
 4661 4784 4786 4684 4633 4656 4703 4741 4612 4796 4757 4701 4659 4657
 4632 4673 4826 4622 4677 4697 4824 4775 4692 4626 4705 4821 4787 4680
 4663 4750 4801]
[4759 4709 4726 4702 4794 4831 4631 4686 4724 4751 4634 4652 4712 4781
 4661 4784 4786 4684 4633 4656 4703 4741 4612 4796 4757 4701 4659 4657
 4632 4673 4826 4622 4677 4697 4824 4775 4692 4626 4705 4821 4787 4680
 4663 4750 4801]
[4759 4709 4726 4702 4794 4831 4631 4686 4724 4751 4634 4652

 5607 5626 5767 5568 5726 5758 5600 5672 5574]
[5666 5739 5598 5586 5642 5678 5689 5632 5624 5801 5682 5793 5708 5778
 5712 5585 5607 5626 5767 5568 5726 5758 5600 5672 5574]
[5708 5778 5712 5585 5607 5626 5767 5568 5726 5758 5600 5672 5574]
56
[14 14 14 14]
14
[5881 5844 5974 5980 6015 6083 5804 5851 5987 5897 5816 6044 5975 5960
 5928 5864 5805 5958 5850 5998 5962 5825 6068 5966 5940 6002 5869 5938
 5895 6064 6046 5978 6020 6051 5846 5855 5823 5879 5860 5909 5948 5902
 5937 5822 5963 5847 5815 5952 5878 5908 5845 5904 6010 5944 5830 5931]
[5881 5844 5974 5980 6015 6083 5804 5851 5987 5897 5816 6044 5975 5960
 5928 5864 5805 5958 5850 5998 5962 5825 6068 5966 5940 6002 5869 5938
 5895 6064 6046 5978 6020 6051 5846 5855 5823 5879 5860 5909 5948 5902
 5937 5822 5963 5847 5815 5952 5878 5908 5845 5904 6010 5944 5830 5931]
[5881 5844 5974 5980 6015 6083 5804 5851 5987 5897 5816 6044 5975 5960
 5928 5864 5805 5958 5850 5998 5962 5825 6068 5966 5940 6002 5869 5938
 5895 6064 6046 5978 6020 

[6443 6473 6337 6442 6344 6397 6439 6369 6438 6446 6409 6470 6353 6458
 6423 6366 6464 6451 6405 6320 6462 6468 6395 6382]
[6438 6446 6409 6470 6353 6458 6423 6366 6464 6451 6405 6320 6462 6468
 6395 6382]
[6464 6451 6405 6320 6462 6468 6395 6382]
88
[22 22 22 22]
22
[6675 6745 6579 6791 6917 6595 6554 6747 6597 6859 6800 6507 6838 6714
 6738 6681 6569 6879 6693 6516 6903 6885 6598 6615 6591 6815 6741 6883
 6542 6716 6665 6723 6649 6531 6476 6682 6678 6646 6913 6633 6540 6820
 6529 6571 6824 6861 6528 6845 6833 6592 6854 6731 6892 6658 6488 6492
 6555 6477 6587 6581 6500 6686 6711 6549 6493 6662 6517 6811 6521 6735
 6904 6804 6680 6768 6900 6644 6596 6749 6613 6663 6480 6860 6862 6905
 6627 6607 6796 6694]
[6675 6745 6579 6791 6917 6595 6554 6747 6597 6859 6800 6507 6838 6714
 6738 6681 6569 6879 6693 6516 6903 6885 6598 6615 6591 6815 6741 6883
 6542 6716 6665 6723 6649 6531 6476 6682 6678 6646 6913 6633 6540 6820
 6529 6571 6824 6861 6528 6845 6833 6592 6854 6731 6892 6658 6488 6492


# 확인하기 : 

In [29]:
from scipy import io
import numpy as np

import copy

In [30]:
mat_file = io.loadmat('groundtruth.mat')

hi_file=io.loadmat('h.mat')

In [38]:
num2=0

f=0
for i in range(len(mat_file['groundtruth']['bboxes'][0])):
    
    if np.any(mat_file['groundtruth']['bboxes'][0][i]-hi_file['groundtruth']['bboxes'][0][i]):
        #print(i)
        num2+=1
        
print(num)
print(num2)

1552
1165


In [36]:
print(hi_file['groundtruth']['bboxes'][0][7819])
print(mat_file['groundtruth']['bboxes'][0][7819])

[[608.5 327.   27.   81. ]]
[[595. 327.  54.  81.]]
